GAN(Generative Adversarial Network)은 생성 모델의 한 종류로, 두 개의 신경망인 생성기(Generator)와 판별기(Discriminator)가 경쟁적으로 학습하는 방식으로 작동합니다.
최근 Image Generation과 같은 Generative model이 급부상하고 있으므로 그 중 가장 간단한 형태인 GAN을 구현해보겠습니다.
이번 예제에서는 MNIST 데이터셋을 이용하여 손글씨 숫자 이미지를 생성하는 GAN을 구현해보겠습니다.

GAN은 생성기(Generator)와 판별기(Discriminator) 두 개의 신경망을 이용합니다.
- 생성기(Generator) : 랜덤한 잡음(Noise)을 입력받아 가짜 이미지를 생성합니다.
- 판별기(Discriminator) : 진짜 이미지와 가짜 이미지를 입력받아 진짜인지 가짜인지 판별합니다.

두 신경망은 경쟁적으로 학습하며, 생성기는 판별기를 속이는 방향으로 학습하고 판별기는 생성기가 만든 가짜 이미지와 진짜 이미지를 잘 판별하도록 학습합니다.
경쟁적으로 학습한다는 것은 생성기가 더 진짜같은 이미지를 생성하려고 노력하고 판별기는 더 정확하게 진짜와 가짜를 판별하려고 노력하는 것을 의미합니다.
결과적으로 생성기는 점점 더 진짜같은 이미지를 생성하게 되고 판별기는 점점 더 정확하게 진짜와 가짜를 판별하게 됩니다.
꽤나 그럴싸한 아이디어죠? 이 아이디어는 2014년 [Ian Goodfellow](https://arxiv.org/abs/1406.2661)가 처음 제안한 것으로, 그 이후로 많은 발전이 있었습니다.
Ian Goodfellow는 딥러닝 분야 과목을 듣게 되면 꼭 한번이상씩은 나오게 되니 잘 기억해두시면 좋을 것 같습니다.

여기서는 최대한 단순하게 아이디어적으로 설명하려고 노력할 것이고, 좀 더 수학적인 증명등과 같은 내용을 보고 싶으시면 위의 논문이나 [다른 설명문](https://process-mining.tistory.com/169)들을 참고하시면 됩니다.
$$
\mathcal{L}_G = -\mathbb{E}_{z \sim p_z(z)} [\log(D(G(z)))] 
$$
$$
\mathcal{L}_D = -\left( \mathbb{E}_{x \sim p_{\text{data}}(x)} [\log(D(x))] + \mathbb{E}_{z \sim p_z(z)} [\log(1 - D(G(z)))] \right) 
$$
가장 간단한 형태의 GAN은 위와 같은 손실함수를 이용하여 학습합니다. 쉽게 말해 둘이 싸운다는 뜻입니다.

필요한 라이브러리를 불러오겠습니다. torch, torchvision, numpy, matplotlib을 사용합니다.

In [ ]:
!pip install torch torchvision numpy matplotlib

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

먼저 데이터셋을 살펴보도록 합시다. 데이터셋은 MNIST 데이터셋을 사용할 것이며, MNIST 데이터셋은 0부터 9까지의 손글씨 숫자 이미지로 구성되어 있습니다.
각 이미지는 28x28 크기의 흑백 이미지로 구성되어 있습니다.

데이터셋을 불러오고, DataLoader를 만들어줍니다. DataLoader는 데이터셋을 batch 단위로 불러오는 역할을 합니다.
도서관에 책을 반납하는 것을 예시로 들면, 한권 가져가서 반납하고, 다시 한권 가져가서 반납하는 것이 아니라 한번에 여러 권을 가져가서 반납하는 것과 같습니다.
이렇게 하면 데이터를 효율적으로 불러오면서 더욱 빠르게 처리할 수 있습니다.

In [ ]:
# MNIST 데이터셋 불러오기

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋을 다운로드 & dataloader 생성
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# 이미지 여러 장 출력하는 함수
def show_images(images, labels):
    images = images / 2 + 0.5 # 이미지를 출력할 수 있게 -1 ~ 1 사이의 값으로 변환
    npimg = images.numpy() # 이미지를 numpy 배열로 변환
    fig, axes = plt.subplots(1, 8, figsize=(12, 12))

    # 이미지 출력
    for i in range(8):
        ax = axes[i]
        ax.imshow(npimg[i].reshape(28, 28), cmap='gray')
        ax.set_title(f'Label: {labels[i].item()}')
        ax.axis('off')
    plt.show()

# 이미지 배치를 불러옴
dataiter = iter(dataloader)
images, labels = dataiter.next()

show_images(images, labels)

위와 같은 데이터로 이루어져있다고 보시면 됩니다. 아주 간단한 데이터셋이지만 GAN을 구현하고 어떻게 학습하는지 알아보는데는 충분합니다.

이제 생성기(Generator)와 판별기(Discriminator)를 만들어보겠습니다.

어떻게 만들어도 상관없지만 가장 단순하게 구현하기 위해 생성기와 판별기는 각각 3개의 레이어로 구성된 MLP(Multi-Layer Perceptron)로 구성하겠습니다.
생성기는 100차원의 랜덤한 잡음을 입력받아 784차원의 가짜 이미지를 출력하도록 구성하겠습니다.
판별기는 784차원의 이미지를 입력받아 1차원의 스칼라값(0이면 가짜, 1이면 진짜인 확률값)을 출력하도록 구성하겠습니다.
왜 굳이 랜덤한 잡음을 입력받은 후에 생성하는지는 여기서 설명하기 너무 곤란하니 추후 딥러닝을 공부하시면서, 또는 관련된 수업을 들으시면서 차차 이해하시면 됩니다.

생성기 판별기는 좀 생소하지만 MLP는 이미 알고 계실 것이라 생각합니다. MLP는 Fully Connected Layer로 구성된 신경망으로, 각 레이어의 모든 뉴런이 이전 레이어의 모든 뉴런과 연결되어 있는 구조입니다.
1학기때 수업에서 배웠으니 괜찮겠죠...?

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


생성기와 판별기를 만들었으니, 이제 학습을 위한 손실함수, 옵티마이저와 hyperparameter를 설정해줍시다.

latent_dim은 생성기의 입력 차원을 의미하며, 생성기는 latent_dim 차원의 랜덤한 잡음을 입력받아 이미지를 생성합니다.
lr은 학습률을 의미하며, 학습률은 학습속도를 조절하는 하이퍼파라미터입니다.

In [ ]:
latent_dim = 100
img_shape = 28 * 28

generator = Generator(input_dim=latent_dim, output_dim=img_shape)
discriminator = Discriminator(input_dim=img_shape)

lr = 0.0002
b1 = 0.5
b2 = 0.999

optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

adversarial_loss = nn.BCELoss()

학습과정이 좀 복잡해 보일 수 있는데, 이해하고 나면 생각보다는 간단합니다.

1. 생성자(generator)는 랜덤한 노이즈를 입력으로 받아 가짜 이미지를 생성합니다.
2. 판별자(discriminator)는 실제 이미지(데이터셋에서 불러온거)와 가짜 이미지를 입력으로 받아 각 이미지가 진짜인지 가짜인지 판별합니다.
3. 생성자(generator)는 판별자(discriminator)를 속이도록 학습합니다. 즉, 생성자(generator)가 만든 가짜 이미지를 판별자(discriminator)가 진짜라고 판별하도록 학습합니다.
4. 판별자(discriminator)는 실제 이미지와 가짜 이미지를 잘 판별하도록 학습합니다.

이 과정을 반복하면 생성자(generator)는 점점 더 진짜같은 가짜 이미지를 생성하게 되고, 판별자(discriminator)는 점점 더 정확하게 진짜와 가짜를 판별하게 됩니다.
그러니까 쉽게 이야기하면 Generator는 데이터셋에서 뽑은 이미지와 비슷한 이미지를 생성하려고 노력하고, Discriminator는 Generator가 만든 이미지와 데이터셋에서 뽑은 이미지를 잘 구분하려고 노력하는 겁니다.

In [ ]:
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator.to(device)
discriminator.to(device)
adversarial_loss.to(device)

for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        
        # 진짠지 가짠지에 대한 레이블 생성
        valid = torch.ones((imgs.size(0), 1), requires_grad=False).to(device)
        fake = torch.zeros((imgs.size(0), 1), requires_grad=False).to(device)
        
        # 진짜 이미지 (데이터셋에서 불러온거)
        real_imgs = imgs.view(imgs.size(0), -1).to(device)
        
        ### 생성기 학습
        optimizer_G.zero_grad()
        
        # 랜덤 노이즈로부터 이미지 생성
        z = torch.randn((imgs.size(0), latent_dim)).to(device)
        gen_imgs = generator(z)
        
        # 생성된 이미지를 판별자에 넣어서 결과 확인
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        

        ### 판별기 학습
        optimizer_D.zero_grad()
        
        # 진짜 이미지 판별
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()
        
        if i % 400 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch {i}/{len(dataloader)} \
                  Loss D: {d_loss.item()}, loss G: {g_loss.item()}")
            
    # 중간 단계 확인을 위해 생성된 이미지 저장
    if epoch % 10 == 0:
        gen_imgs = gen_imgs.view(gen_imgs.size(0), 1, 28, 28)
        save_image(gen_imgs.data[:25], f"images/{epoch}.png", nrow=5, normalize=True)


이제 학습이 완료되었으니 결과를 확인해보겠습니다. 생성기(generator)가 만든 가짜 이미지를 확인해보겠습니다.

In [ ]:
def show_generated_imgs(generator, latent_dim, num_images=5):
    z = torch.randn(num_images, latent_dim).to(device)
    gen_imgs = generator(z)
    gen_imgs = gen_imgs.view(gen_imgs.size(0), 1, 28, 28)
    gen_imgs = gen_imgs.detach().cpu().numpy()

    fig, axes = plt.subplots(1, num_images, figsize=(num_images, 1))
    for i in range(num_images):
        axes[i].imshow(np.transpose(gen_imgs[i], (1, 2, 0)).squeeze(), cmap='gray')
        axes[i].axis('off')
    plt.show()

show_generated_imgs(generator, latent_dim)

이렇게 생각보다는 간단하게(?) GAN을 구현해보았습니다. GAN은 처음에 이해하기 어려울 수 있지만, 한번 구현해보면 생각보다는 간단하다는 것을 느낄 수 있습니다.
GAN은 2014년에 처음 제안된 이후로 많은 발전이 있었고, 다양한 변형이 나오고 있습니다. 이번 예제는 그 중에서도 가장 간단한 형태의 GAN을 구현해보았습니다.

좀 관심이 생기셨다면(...) [DCGAN](https://arxiv.org/abs/1511.06434), [WGAN](https://arxiv.org/abs/1701.07875) 등 다양한 GAN을 찾아보시면 좋을 것 같습니다.
이미지 생성에 관심이 생기셨다면 [DDPM](https://arxiv.org/abs/2006.11239), [VQ-VAE](https://arxiv.org/abs/1711.00937) 등 다양한 모델을 찾아보시면 좋을 것 같습니다.
매우 어려울 수도 있긴 하지만 그걸 극복하고 이해하고 나면 정말 재밌는 분야라고 생각합니다.